`preprocessing_notebook_part_1.ipynb`: outputs a structured parquet format and saves it.

**The purpose of this project is to analyse the location a taxi driver should work to make the most profit.**

I will analyse, if possible, all yellow, green, and FHV taxis, compare them and draw conclusions with why they might be the same or different. Obviously there will be obstacles, but I'll just sort of wing it and solve every roadblocks or problems that might occur throughout the project.

(I hope I can stick with this idea and I don't have to change topics in the middle of doing it)

In [1]:
import sys
sys.path.insert(0, '../scripts')

from readparquet import read_parquet
sdf = read_parquet('../data/raw/fhvhv_tripdata_2022-01.parquet')
# convert to pandas dataframe for visualisation
df = sdf.sample(0.001, seed=0).toPandas()

22/07/29 14:24:57 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 192.168.207.1 instead (on interface eth1)
22/07/29 14:24:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/29 14:24:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df.info()
df.sample(n=5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14731 entries, 0 to 14730
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hvfhs_license_num     14731 non-null  object        
 1   dispatching_base_num  14731 non-null  object        
 2   originating_base_num  10793 non-null  object        
 3   request_datetime      14731 non-null  datetime64[ns]
 4   on_scene_datetime     10795 non-null  datetime64[ns]
 5   pickup_datetime       14731 non-null  datetime64[ns]
 6   dropoff_datetime      14731 non-null  datetime64[ns]
 7   PULocationID          14731 non-null  int64         
 8   DOLocationID          14731 non-null  int64         
 9   trip_miles            14731 non-null  float64       
 10  trip_time             14731 non-null  int64         
 11  base_passenger_fare   14731 non-null  float64       
 12  tolls                 14731 non-null  float64       
 13  bcf             

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
3407,HV0005,B03406,None,2022-01-09 08:51:45,NaT,2022-01-09 08:57:50,2022-01-09 09:04:00,229,237,0.834,...,0.49,2.75,0.0,0.00,5.47,N,N,N,N,N
13108,HV0003,B03404,B03404,2022-01-29 01:16:37,2022-01-29 01:19:22,2022-01-29 01:19:54,2022-01-29 01:58:29,42,121,18.670,...,4.48,0.00,0.0,0.00,39.97,N,N,,N,N
2082,HV0005,B03406,None,2022-01-06 10:05:52,NaT,2022-01-06 10:09:02,2022-01-06 10:16:32,36,80,1.772,...,0.79,0.00,0.0,0.00,6.31,N,N,N,N,Y
1957,HV0005,B03406,None,2022-01-06 04:30:58,NaT,2022-01-06 04:36:37,2022-01-06 04:52:26,189,17,2.611,...,1.27,0.00,0.0,0.00,10.84,N,N,N,N,N
7774,HV0003,B03404,B03404,2022-01-18 09:29:58,2022-01-18 09:35:58,2022-01-18 09:35:58,2022-01-18 10:00:19,138,87,11.560,...,3.25,2.75,2.5,4.37,28.46,N,N,,N,N


Hmmmm, it seems like there are too many features necessary for data analysis. Let's remove the ones I am certain are completely useless.

In [11]:
from pyspark.sql.functions import col
sdf_s = sdf.select(col("pickup_datetime"), col("dropoff_datetime"), col("PULocationID"), col("DOLocationID"), \
    col("trip_miles"), col("base_passenger_fare"), col("driver_pay"), col("tips"))
df_s = sdf_s.sample(0.001, seed=0).toPandas()
df_s.sample(n=5)
#sdf.drop("dispatching_base_num").drop("originating_base_num")

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,base_passenger_fare,driver_pay,tips
3745,2022-01-10 03:15:10,2022-01-10 03:41:54,153,51,6.850,25.30,22.85,0.0
12996,2022-01-28 21:03:59,2022-01-28 21:13:17,210,155,1.880,10.99,6.74,0.0
10699,2022-01-24 00:17:34,2022-01-24 00:37:45,134,171,7.580,24.42,18.50,0.0
5786,2022-01-14 21:39:12,2022-01-14 21:54:41,198,102,1.923,12.85,12.64,0.0
1139,2022-01-04 04:05:52,2022-01-04 04:22:49,243,168,6.330,20.82,15.49,0.0


The idea is that, essentially, only the PU location ID will be taken into account for a single taxi ride. (Why not DO?)